In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
import nltk
from nltk.corpus import stopwords

In [8]:
yelp = pd.read_csv('yelp.csv')

In [9]:
yelp.shape


(10000, 10)

In [19]:
yelp.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [10]:
#making a dataset for predicting if a review is good or bad
yelp_class = yelp[(yelp['stars'] == 1) | (yelp['stars'] == 5)]
yelp_class.shape

(4086, 10)

In [11]:
X = yelp_class['text']
y = yelp_class['stars']

In [21]:
import string
def text_process(text):
   
    

    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [13]:
sample_text = "Hey there! This is a sample review, which happens to contain punctuations."
print(text_process(sample_text))



['Hey', 'sample', 'review', 'happens', 'contain', 'punctuations']


In [24]:
text_process(X)

['wife',
 'took',
 'birthday',
 'breakfast',
 'excellent.',
 'weather',
 'perfect',
 'made',
 'sitting',
 'outside',
 'overlooking',
 'grounds',
 'absolute',
 'pleasure.',
 'waitress',
 'excellent',
 'food',
 'arrived',
 'quickly',
 'semi-busy',
 'Saturday',
 'morning.',
 'looked',
 'like',
 'place',
 'fills',
 'pretty',
 'quickly',
 'earlier',
 'get',
 'better.',
 'favor',
 'get',
 'Bloody',
 'Mary.',
 'phenomenal',
 'simply',
 'best',
 "I've",
 'ever',
 'had.',
 "I'm",
 'pretty',
 'sure',
 'use',
 'ingredients',
 'garden',
 'blend',
 'fresh',
 'order',
 'it.',
 'amazing.',
 'EVERYTHING',
 'menu',
 'looks',
 'excellent,',
 'white',
 'truffle',
 'scrambled',
 'eggs',
 'vegetable',
 'skillet',
 'tasty',
 'delicious.',
 'came',
 '2',
 'pieces',
 'griddled',
 'bread',
 'amazing',
 'absolutely',
 'made',
 'meal',
 'complete.',
 'best',
 '"toast"',
 "I've",
 'ever',
 'had.',
 'Anyway,',
 "can't",
 'wait',
 'go',
 'back!I',
 'idea',
 'people',
 'give',
 'bad',
 'reviews',
 'place.',
 'goes',

In [14]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(X)

In [15]:
len(bow_transformer.vocabulary_)

26435

In [16]:
review_25 = X[24]
review_25

"I love this place! I have been coming here for ages.\nMy favorites: Elsa's Chicken sandwich, any of their burgers, dragon chicken wings, china's little chicken sandwich, and the hot pepper chicken sandwich. The atmosphere is always fun and the art they display is very abstract but totally cool!"

In [26]:
bow_25 = bow_transformer.transform([review_25])
print(bow_25)

  (0, 2099)	1
  (0, 3006)	1
  (0, 8909)	1
  (0, 9151)	1
  (0, 9295)	1
  (0, 9616)	1
  (0, 9727)	1
  (0, 10847)	1
  (0, 11443)	3
  (0, 11492)	1
  (0, 11878)	1
  (0, 12221)	1
  (0, 13323)	1
  (0, 13520)	1
  (0, 14481)	1
  (0, 15165)	1
  (0, 16379)	1
  (0, 17812)	1
  (0, 17951)	1
  (0, 20044)	1
  (0, 20298)	1
  (0, 22077)	3
  (0, 24797)	1
  (0, 26102)	1


In [27]:
X = bow_transformer.transform(X)

In [30]:
#using 30% of the dataset for testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [31]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [32]:
preds = nb.predict(X_test)

In [33]:
from sklearn.metrics import confusion_matrix, classification_report
#Accuracy measure
print(confusion_matrix(y_test, preds))
print('\n')
print(classification_report(y_test, preds))

[[157  71]
 [ 24 974]]


             precision    recall  f1-score   support

          1       0.87      0.69      0.77       228
          5       0.93      0.98      0.95       998

avg / total       0.92      0.92      0.92      1226



In [36]:
positive_review_test = yelp_class['text'][59]
positive_review_test
#this is a sample test to see if our predictor works correctly

"This restaurant is incredible, and has the best pasta carbonara and the best tiramisu I've had in my life. All the food is wonderful, though. The calamari is not fried. The bread served with dinner comes right out of the oven, and the tomatoes are the freshest I've tasted outside of my mom's own garden. This is great attention to detail.\n\nI can no longer eat at any other Italian restaurant without feeling slighted. This is the first place I want take out-of-town visitors I'm looking to impress.\n\nThe owner, Jon, is helpful, friendly, and really cares about providing a positive dining experience. He's spot on with his wine recommendations, and he organizes wine tasting events which you can find out about by joining the mailing list or Facebook page."

In [37]:
#it is classified as a positive review (5 star)
positive_review_transformed = bow_transformer.transform([positive_review_test])
nb.predict(positive_review_transformed)[0]

5

In [38]:
#we use our predictor to classify a general review, we dont know if it will be positive or negative
general_review_test = yelp_class['text'][79]
general_review_test
#but once we print the review we find an underlying negative tone, lets see if our classifier works

'This is my first year participating in Arizona to sell clothing, just wandering how busy its goona be in there ??? little worried, can any one reply me ??'

In [39]:
general_review_transformed = bow_transformer.transform([general_review_test])
nb.predict(general_review_transformed)[0]
#this review has been classified as 1 , it works (negative)

1

In [40]:
#we repeat the test
general_review_test1 = yelp_class['text'][281]
general_review_test1

'Still quite poor both in service and food. maybe I made a mistake and ordered Sichuan Gong Bao ji ding for what seemed like people from canton district. Unfortunately to get the good service U have to speak Mandarin/Cantonese. I do speak a smattering but try not to use it as I never feel confident about the intonation. \n\nThe dish came out with zichini and bell peppers (what!??)  Where is the peanuts the dried fried red peppers and the large pieces of scallion. On pointing this out all I got was " Oh you like peanuts.. ok I will put some on" and she then proceeded to get some peanuts and sprinkle it on the chicken.\n\nWell at that point I was happy that atleast the chicken pieces were present else she would probably end up sprinkling raw chicken pieces on it like the raw peanuts she dumped on top of the food. \n\nWell then  I spoke a few chinese words and the scowl turned into a smile and she then became a bit more friendlier. \n\nUnfortunately I do not condone this type of behavior.

In [41]:
general_review_transformed = bow_transformer.transform([general_review_test1])
nb.predict(general_review_transformed)[0]
#again we get that this is a negative review -- classified as 1

1